In [4]:
!pip uninstall -y pyarrow datasets

Found existing installation: pyarrow 15.0.2
Uninstalling pyarrow-15.0.2:
  Successfully uninstalled pyarrow-15.0.2
Found existing installation: datasets 2.19.0
Uninstalling datasets-2.19.0:
  Successfully uninstalled datasets-2.19.0


In [5]:
!pip install --no-use-pep517 pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 62.0 MB/s eta 0:00:00:00:010:01m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.11.3 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.0.0 which is incompatible.


In [6]:
!pip install datasets

  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.0-py3-none-any.whl (542 kB)


In [8]:
from datasets import load_dataset

dataset = load_dataset("Hello-SimpleAI/HC3", name="all")

Generating train split:   0%|          | 0/24322 [00:00<?, ? examples/s]